# Redes Multicapa de Aerolíneas

Procesamiento de datos de rutas aéreas por distintas aerolíneas en R.

In [1]:
# Librerías necesarias
library(tidyverse)
library(data.table)
library(igraph)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.0       ✔ purrr   0.3.2  
✔ tibble  2.1.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

The following object is masked from ‘package:purrr’:

    transpose


Attaching package: ‘igraph’

The following objects are masked from ‘package:dplyr’:

    as_data_frame, groups, union

The following objects are masked from ‘package:purrr’:

    compose, simplify

The following object is masked from ‘package:tidyr’:

    crossing

The following object is masked from ‘package:tibble’:

    as_data_frame

The following objects are masked from ‘package:stats’:

    decompose, spec

In [2]:
# Se carga la tabla de rutas aéreas
tabla_rutas <- data.table::fread(
  input = ""
)

ERROR: Error in data.table::fread(input = ""): Input is empty or only contains BOM or terminal control characters


In [ ]:
# Se carga la tabla de aerolíneas
tabla_aerolineas <- data.table::fread(
  input = ""
  )

In [ ]:
# Se carga la tabla de aeropuertos
tabla_aeropuertos <- data.table::fread(
  input = ""
  )

In [ ]:
# Se muestra la información que contienen (6 primeros renglones)
tabla_rutas %>% head
tabla_aerolineas %>% head
tabla_aeropuertos %>% head

In [ ]:
# Renombramos las columnas de la tabla de aerolíneas
names(tabla_aerolineas) <- c("Airline ID", 
                             "Name", 
                             "Alias", 
                             "IATA", 
                             "ICAO", 
                             "Callsign", 
                             "Country", 
                             "Active"
                              )

In [ ]:
# Checamos la tabla_aerolíneas con los nuevos nombres
tabla_aerolineas %>% head  

In [ ]:
# Quitamos el primer renglón de la tabla_aerolíneas
tabla_aerolineas <- tabla_aerolineas [-c(1), ]

In [ ]:
#Checamos la tabla_aerolíneas
tabla_aerolineas %>% head 

In [ ]:
#Quitamos columnas de ID que no necesitamos de tabla_rutas
tabla_rutas <- tabla_rutas %>%
  dplyr::select(Airline,
                Source_airport,
                Destination_Airport,
                Codeshare,
                Stops_Number,
                Equipment)

In [ ]:
#Checamos la tabla_rutas
tabla_rutas %>% head

In [ ]:
#Ahora hacemos un ciclo que recorra toda la tabla_aerolíneas, el cual 
#remueva los renglones correspondientes a las aerolíneas que NO están activas.

tabla_aerolineas <- tabla_aerolineas %>% 
  dplyr::filter(Active == "Y")

In [ ]:
#Checamos la tabla_aerolineas
tabla_aerolineas %>% head

In [ ]:
#Falta remover los renglones de la tabla_aeropuertos que contienen algún aeropuerto inactivo

In [ ]:
#Checamos la tabla_aeropuertos
tabla_aeropuertos %>% head

In [ ]:
#Encontramos la intersección entre tabla_aerolineas y tabla_rutas
#para usar sólo las rutas de las aerolíneas activas.
tabla_rutas <- semi_join(tabla_rutas, tabla_aerolineas, by = c("Airline" = "IATA"))

In [ ]:
#Encontramos la intersección entre tabla_aeropuertos y tabla_rutas
#para usar sólo las rutas de las aeropuertos activos.

In [ ]:
#Checamos la tabla_rutas modificada
tabla_rutas %>% head

In [ ]:
#Cargamos la tabla de rutas2 (actualizada en dic de 2018)
tabla_rutas2 <- data.table::fread(
  input = ""
)

In [ ]:
#Checamos la tabla_rutas2
tabla_rutas2 %>% head

In [ ]:
#Quitamos el primer y segundo renglón de la tabla_aerolíneas
tabla_rutas2 <- tabla_rutas2 [-c(2), ]

In [ ]:
#Checamos tabla_rutas2
tabla_rutas2 %>% head

In [ ]:
# Renombramos las columnas de la tabla_rutas2
names(tabla_rutas2) <- c("Airline", 
                         "Source", 
                         "Destination", 
                         "Codeshare", 
                         "Stops", 
                         "Equipment"
)

In [ ]:
#Checamos tabla_rutas2
tabla_rutas2 %>% head

In [ ]:
#Encontramos la intersección entre tabla_aerolineas y tabla_rutas2
#para usar sólo las rutas de las aerolíneas activas.
tabla_rutas2 <- semi_join(tabla_rutas2, tabla_aerolineas, by = c("Airline" = "IATA"))

Puesto que tabla_rutas2 contiene menos datos, usaremos esos...

In [ ]:
#Queremos converir la columna de Codeshare a booleano
tabla_rutas2 <- tabla_rutas2 %>%
  dplyr::mutate(Codeshare = ifelse(test = Codeshare == "*",
                                   yes = TRUE,
                                   no = FALSE
  )
  )

In [ ]:
#Checamos
tabla_rutas2 %>% head

In [ ]:
#Ahora voy a quitar las rutas que contienen escalas, 
#i.e. los que son distintos de 0 en "Stops_Number"
tabla_rutas2 <- tabla_rutas2 %>% 
  dplyr::filter(Stops == 0)

In [ ]:
#Checamos
tabla_rutas2

In [ ]:
#Ahora voy a quitar las rutas que contienen Codeshare, 
#i.e. los que son TRUE en "Codeshare"
tabla_rutas2 <- tabla_rutas2 %>%
  dplyr::filter(Codeshare == FALSE)

In [ ]:
#Checamos
tabla_rutas2

In [ ]:
#Cambio orden de columnas
tabla_rutas2[ , -1]
tabla_rutas2[ , 1]

da.fr <- cbind(tabla_rutas2[,-1], 
               Airline = tabla_rutas2$Airline
)

In [ ]:
#Checo cómo se ve
da.fr %>% head

In [ ]:
#Hago red
g <- igraph::graph_from_data_frame(d = da.fr, 
                                   directed = TRUE)

Se grafican las rutas aéreas de todas las aerolíneas como una red monocapa.

In [ ]:
#Graficamos todo en una capa
plot(g, edge.arrow.size=.4, edge.curved=.1,
     vertex.color="orange", 
     vertex.frame.color="#555555",
     vertex.label.color="black",
     vertex.label.cex=.7)

Medidas de centralidad de esta red monocapa de rutas.

In [ ]:
#Grado
degree(g) %>% head

In [ ]:
#Distribución de grado


In [ ]:
#Estructura de degree(g)
str(degree(g))

In [ ]:
#Average Shortest Path Length
average.path.length(g)

In [ ]:
#Betweenness
betweenness(g) %>% head

In [ ]:
#Diámetro
diameter(g)

In [ ]:
#Radio
radius(g)

In [ ]:
#Excentricidad
eccentricity(g) %>% head

In [ ]:
#Transitivity/Clustering Coefficient
transitivity(g)

Para crear la red multicapa

In [ ]:
#Vemos cuántas aerolíneas distintas tenemos
length(unique(da.fr$Airline))

In [ ]:
#Tomo todas las aerolíneas posibles con unique
#Como las aerolíneas son de tipo Factor, lo convierto a caracteres 
mis_aerolineas <- as.character(unique(da.fr$Airline)) 

In [ ]:
#La lista "hereda" los nombres
names(mis_aerolineas) <- mis_aerolineas

In [ ]:
#Se aplica el filtrado para aerolíneas, que corresponderán a distintas capas 
#y le agrega la gráfica a cada capa
graph_capas <- lapply(mis_aerolineas, FUN = function(i){
  xf <- dplyr::filter(.data = da.fr, Airline == i)
  g <- igraph::graph_from_data_frame(d = xf, 
                                     directed = TRUE)
}) 

In [ ]:
#Eg. Se grafica la red de la capa correspondiente a la aerolínea 2B
plot(graph_capas$`2B`)

In [ ]:
#Eg. Se grafica la red de la capa correspondiente a la aerolínea AM
plot(graph_capas$AM)

In [ ]:
#Agrego al ciclo las centralidades de cada capa
capas <- lapply(mis_aerolineas, FUN = function(i){
  xf <- dplyr::filter(.data = da.fr, Airline == i)
  g <- igraph::graph_from_data_frame(d = xf, 
                                     directed = TRUE)
  Resultados <- list(APL = average.path.length(g), 
                     Diametro = diameter(g),
                     Radio = radius(g),
                     Clustering_Coefficient = transitivity(g),
                     Red = g)
  
  return(Resultados)
}) 

In [ ]:
#Eg. Llamamos la información de APL de la capa correspondiente a la aerolínea 2B
capas$`2B`$APL

In [ ]:
#Eg. Llamamos la información de APL de la capa correspondiente a la aerolínea AM
capas$AM$APL

In [ ]:
#Eg. Llamamos la información de APL de la capa correspondiente a la aerolínea AM
capas$AM$Diametro

In [ ]:
#Eg. Llamamos la información de APL de la capa correspondiente a la aerolínea AM
capas$AM$Radio

In [ ]:
#Eg. Llamamos la información de APL de la capa correspondiente a la aerolínea AM
capas$AM$Clustering_Coefficient

In [ ]:
#Eg. Para graficar la capa correspondiente a la aerolínea 2I.
#Graficamos la entrada Red de la aerolínea 2I
plot(capas$`2I`$Red)

In [ ]:
#Juntamos toda la información de las capas
capas_totales <- capas[1:567
                 ]

In [ ]:
#Checamos los nombres de todas las aerolíneas
names(capas_totales)

In [ ]:
#Generalizamos.
#Hacemos un ciclo que llame a la función capas y las grafique
for(i in seq_along(capas)){
  plot(capas[[i]][["Red"]],
       main = names(capas[i]))
}

In [ ]:
#Eg. Llamamos la entrada 1 del conjunto de capas totales y después llamamos la Red
capas[[1]][["Red"]]

In [ ]:
#Para graficarlo
plot(capas_totales[[1]][["Red"]])

In [ ]:
#Para averiguar cuántos nodos tenemos en la tabla_rutas2